# Glean‑Style Multi‑Tool Agent with LangChain + LangGraph (Mock Demo)

This notebook shows how to build a **Glean‑style enterprise assistant** using:

- **LangChain** for LLMs and prompts  
- **LangGraph** for multi‑step / multi‑agent orchestration  
- **Mock tools** that simulate:
  - Google Docs
  - Gmail
  - Slack
  - Jira
  - Local knowledge base

No real APIs are called – everything is **in‑memory mock data**, so you can run this safely without credentials.

---

## High‑Level Architecture

We’ll build a simple agent workflow:

1. **Router Agent (LLM)**  
   - Looks at the user question  
   - Decides which “tool” is most relevant (google_docs / gmail / slack / jira / local_kb)  
   - Outputs a small JSON with `tool` and `tool_query`

2. **Tool Executor Node (Python)**  
   - Calls the selected tool function on our mock data  
   - Returns a `tool_result` string

3. **Answer Agent (LLM)**  
   - Takes `question + tool_result`  
   - Generates a nice, user‑friendly answer

All of this is wired together with **LangGraph**.


## 1. Install Dependencies (if needed)

Run this cell once to install the necessary libraries.

> 💡 If you already installed these in your environment, you can skip this or ignore any "already satisfied" messages.


In [ ]:
# Install core libs for this notebook.
# You can comment this out if everything is already installed.

#%pip install -q langchain langchain-core langchain-community langchain-openai langgraph langchain-text-splitters

## 2. Imports & LLM Setup

We will use **OpenAI** via `langchain-openai`.  
You need an OpenAI API key (`OPENAI_API_KEY`).


In [5]:
from typing import TypedDict, Any, List, Dict
import json
from dataclasses import dataclass

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END

# 👉 Put your OpenAI API key here (or use environment variable if you prefer)
OPENAI_API_KEY = "sk-REPLACE_ME"

# Create a small, cheap LLM for routing and answering.
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=OPENAI_API_KEY,
)

print("LLM ready.")

LLM ready.


## 3. Mock Data Sources (Fake Glean‑Style Backends)

Instead of calling real Slack / Jira / Gmail / Google Docs APIs,  
we’ll create **in‑memory dictionaries and lists** that simulate each system.

This keeps the notebook fully self‑contained and safe.


In [6]:
# --- Mock Google Docs ---
MOCK_GOOGLE_DOCS: Dict[str, str] = {
    "doc_resume": """
Saikrishna Vinjamuri is a Senior Data Engineer and GenAI developer.
Tech stack: Python, Java, AWS, Glue, Lambda, RDS, DynamoDB, Kafka, Spark,
LangChain, LangGraph, OpenAI, Cohere, Pinecone, Snowflake, dbt, Airflow.
""",

    "doc_policy": """
Remote Work Policy:
Employees may work remotely up to 3 days per week.
All remote work must comply with data security and VPN usage standards.
"""
}

# --- Mock Gmail inbox ---
MOCK_GMAIL: List[Dict[str, str]] = [
    {
        "from": "manager@example.com",
        "subject": "Deployment delayed",
        "body": "The production deployment is delayed due to failing integration tests."
    },
    {
        "from": "recruiter@example.com",
        "subject": "Interview schedule",
        "body": "We would like to schedule a technical interview next Tuesday."
    },
]

# --- Mock Slack messages ---
MOCK_SLACK_CHANNELS: Dict[str, List[str]] = {
    "#eng-team": [
        "We need to add monitoring for the new feature store pipeline.",
        "Reminder: standup is at 10 AM CST.",
    ],
    "#genai-poc": [
        "The LangGraph-based Glean-style agent is almost ready.",
        "Need to add Jira integration and Gmail summarization.",
    ],
}

# --- Mock Jira issues ---
MOCK_JIRA_ISSUES: List[Dict[str, str]] = [
    {
        "key": "ENG-101",
        "summary": "Create RAG pipeline for internal docs",
        "status": "In Progress"
    },
    {
        "key": "ENG-102",
        "summary": "Fix Glue job for feature store exports",
        "status": "To Do"
    },
]

# --- Mock Local Knowledge Base ---
MOCK_LOCAL_DOCS: List[str] = [
    "Feature store SDK enables ML engineers to register and query features.",
    "LangGraph is used to orchestrate agents and tools in GenAI systems.",
    "Snowflake, Kafka, and dbt are core components of the data platform.",
]

print("Mock data sources initialized.")

Mock data sources initialized.


## 4. Define Tool Functions (Pure Python)

Each tool is a **normal Python function** that knows how to query one data source.

The LLM will not call them directly using the built‑in OpenAI tools mechanism –  
instead, we will:

1. Ask an LLM **which tool to use** (router step)  
2. Call the selected tool from Python (LangGraph node)  
3. Then pass the tool result back into another LLM to generate the final answer  


In [7]:
def search_google_docs(query: str) -> str:
    """Simulate semantic search over mock Google Docs (very simple contains filter)."""
    query_lower = query.lower()
    hits = []
    for doc_id, content in MOCK_GOOGLE_DOCS.items():
        if query_lower in content.lower():
            hits.append(f"[DOC: {doc_id}]\n{content.strip()}")
    if not hits:
        return "No matching content found in Google Docs."
    return "\n\n---\n\n".join(hits)


def search_gmail(query: str) -> str:
    """Simulate search over mock Gmail messages."""
    query_lower = query.lower()
    hits = []
    for msg in MOCK_GMAIL:
        blob = f"From: {msg['from']}\nSubject: {msg['subject']}\nBody: {msg['body']}"
        if query_lower in blob.lower():
            hits.append(blob)
    if not hits:
        return "No matching emails found in Gmail."
    return "\n\n---\n\n".join(hits)


def search_slack(query: str) -> str:
    """Simulate search over mock Slack channels."""
    query_lower = query.lower()
    hits = []
    for channel, messages in MOCK_SLACK_CHANNELS.items():
        for m in messages:
            if query_lower in m.lower():
                hits.append(f"[Channel {channel}] {m}")
    if not hits:
        return "No matching Slack messages found."
    return "\n".join(hits)


def search_jira(query: str) -> str:
    """Simulate search over mock Jira issues by summary / key."""
    query_lower = query.lower()
    hits = []
    for issue in MOCK_JIRA_ISSUES:
        blob = f"{issue['key']}: {issue['summary']} (Status: {issue['status']})"
        if query_lower in blob.lower():
            hits.append(blob)
    if not hits:
        return "No matching Jira issues found."
    return "\n".join(hits)


def search_local_kb(query: str) -> str:
    """Simulate search over a small local knowledge base list."""
    query_lower = query.lower()
    hits = []
    for doc in MOCK_LOCAL_DOCS:
        if query_lower in doc.lower():
            hits.append(doc)
    if not hits:
        return "No relevant local knowledge found."
    return "\n".join(hits)


# A registry so we can map tool names (strings) to functions
TOOL_REGISTRY = {
    "google_docs": search_google_docs,
    "gmail": search_gmail,
    "slack": search_slack,
    "jira": search_jira,
    "local_kb": search_local_kb,
}

print("Tools registered:", list(TOOL_REGISTRY.keys()))

Tools registered: ['google_docs', 'gmail', 'slack', 'jira', 'local_kb']


## 5. Router Agent (LLM)

The **Router Agent** decides which tool to use based on the user question.

We’ll prompt the LLM to output a **small JSON** with two fields:

- `tool` → one of: `google_docs`, `gmail`, `slack`, `jira`, `local_kb`  
- `tool_query` → what we should search for inside that tool

Example output:

```json
{"tool": "gmail", "tool_query": "deployment"}
```

We’ll parse this JSON in Python and pass it into the Tool Executor.


In [8]:
ROUTER_SYSTEM_PROMPT = """
You are a routing assistant for an enterprise knowledge agent.

Your job:
- Look at the user's question
- Decide which ONE data source is best to answer it:
    - `google_docs`  -> if question is about resumes, policies, docs content
    - `gmail`        -> if question is about emails
    - `slack`        -> if question is about team chats or channels
    - `jira`         -> if question is about tasks, tickets, issues, work items
    - `local_kb`     -> if question is about internal platform / architecture notes

Respond ONLY with a valid JSON object with keys:
- "tool": one of ["google_docs", "gmail", "slack", "jira", "local_kb"]
- "tool_query": a short text query to search for in that tool
"""


def router_agent(question: str) -> Dict[str, str]:
    """Call LLM to decide which tool to use and what query to run."""
    messages = [
        SystemMessage(content=ROUTER_SYSTEM_PROMPT),
        HumanMessage(content=question),
    ]
    resp = llm.invoke(messages)
    raw = resp.content.strip()
    # Try to parse JSON; if it fails, fall back to a default
    try:
        parsed = json.loads(raw)
        tool = parsed.get("tool", "local_kb")
        tool_query = parsed.get("tool_query", question)
    except Exception:
        tool, tool_query = "local_kb", question

    return {"tool": tool, "tool_query": tool_query}


# Quick smoke test of router:
print(router_agent("Summarize my resume tech stack from Google Docs."))
print(router_agent("Any Jira tickets about feature store?"))
print(router_agent("What emails mention deployment?"))

{'tool': 'google_docs', 'tool_query': 'resume tech stack'}
{'tool': 'jira', 'tool_query': 'feature store'}
{'tool': 'gmail', 'tool_query': 'deployment'}


## 6. Define LangGraph State and Nodes

We’ll use a small state dictionary passed between nodes:

```python
class AgentState(TypedDict):
    question: str
    tool: str
    tool_query: str
    tool_result: str
    answer: str
```

### Nodes

1. `route_node`  
   - Uses `router_agent` to set `tool` + `tool_query`

2. `tool_node`  
   - Looks up the Python function for `tool`  
   - Calls it with `tool_query`  
   - Stores the result in `tool_result`

3. `answer_node`  
   - Calls LLM with `question` + `tool_result`  
   - Produces final `answer`


In [11]:
class AgentState(TypedDict, total=False):
    question: str
    tool: str
    tool_query: str
    tool_result: str
    answer: str


def route_node(state: AgentState) -> AgentState:
    """LangGraph node: decide which tool to call."""
    question = state["question"]
    routing = router_agent(question)
    return {
        "tool": routing["tool"],
        "tool_query": routing["tool_query"],
    }


def tool_node(state: AgentState) -> AgentState:
    """LangGraph node: execute the selected tool on the tool_query."""
    tool_name = state.get("tool", "local_kb")
    tool_query = state.get("tool_query", state["question"])
    tool_fn = TOOL_REGISTRY.get(tool_name)

    if tool_fn is None:
        result = f"Unknown tool '{tool_name}'. No result."
    else:
        result = tool_fn(tool_query)

    return {"tool_result": result}


ANSWER_SYSTEM_PROMPT = """
You are an enterprise assistant (Glean-style).

You receive:
- The user's original question
- Results from one backend tool (Google Docs / Gmail / Slack / Jira / local KB)

Your job:
- Use the tool results to answer clearly and concisely.
- If the tool result says there were no matches, say that honestly.
- If relevant, mention which source you used (Docs, Gmail, Slack, Jira, Local KB).
"""


def answer_node(state: AgentState) -> AgentState:
    """LangGraph node: generate final answer using question + tool_result."""
    question = state["question"]
    tool_name = state.get("tool", "local_kb")
    tool_result = state.get("tool_result", "")

    messages = [
        SystemMessage(content=ANSWER_SYSTEM_PROMPT),
        HumanMessage(
            content=(
                f"User question: {question}\n\n"
                f"Tool used: {tool_name}\n\n"
                f"Tool result:\n{tool_result}"
            )
        ),
    ]
    resp = llm.invoke(messages)
    return {"answer": resp.content}

## 7. Build and Compile the LangGraph

Now we wire the nodes together into a simple linear graph:

```text
route_node → tool_node → answer_node → END
```


In [12]:
graph = StateGraph(AgentState)

graph.add_node("route", route_node)
graph.add_node("tool", tool_node)
graph.add_node("answer", answer_node)

graph.set_entry_point("route")
graph.add_edge("route", "tool")
graph.add_edge("tool", "answer")
graph.add_edge("answer", END)

app = graph.compile()
print("Graph compiled.")

Graph compiled.


## 8. Try Some Example Queries

Now we can send questions into the compiled graph with `app.invoke(...)`.

The state we pass in only needs the `question` –  
the rest will be filled step‑by‑step by our nodes.


In [13]:
def run_query(question: str):
    print("""
==============================
QUESTION:
==============================""")
    print(question)
    print("\nRunning agent...\n")

    result_state = app.invoke({"question": question})
    print("""
==============================
FINAL ANSWER:
==============================""")
    print(result_state["answer"])


# --- Examples ---
run_query("List the main technologies in my resume from Google Docs.")
run_query("Any Jira tickets that talk about feature store or Glue?")
run_query("What Slack messages mention LangGraph?")
run_query("Do we have any emails about deployment delays?")
run_query("What internal notes discuss the data platform components?")


QUESTION:
List the main technologies in my resume from Google Docs.

Running agent...


FINAL ANSWER:
It appears that there are no matching technologies listed in your resume from Google Docs.

QUESTION:
Any Jira tickets that talk about feature store or Glue?

Running agent...


FINAL ANSWER:
There are no matching Jira tickets that discuss feature store or Glue.

QUESTION:
What Slack messages mention LangGraph?

Running agent...


FINAL ANSWER:
I found a Slack message that mentions LangGraph in the channel #genai-poc: "The LangGraph-based Glean-style agent is almost ready."

QUESTION:
Do we have any emails about deployment delays?

Running agent...


FINAL ANSWER:
There are no emails found regarding deployment delays in Gmail.

QUESTION:
What internal notes discuss the data platform components?

Running agent...


FINAL ANSWER:
I couldn't find any internal notes discussing the data platform components in the local knowledge base.


## 9. Next Steps / How to Extend This

You now have a **Glean‑style multi‑tool agent** with:

- LangChain LLM calls
- LangGraph orchestration
- Router → Tool → Answer pattern
- Mock tools for:
  - Google Docs
  - Gmail
  - Slack
  - Jira
  - Local knowledge base

### Ideas to extend:
- Replace mock tools with REAL APIs:
  - Google Docs API
  - Gmail API
  - Slack Web API
  - Jira Cloud REST API
- Add **RAG** using vector stores (FAISS, Chroma) instead of simple `in` matching
- Add more tools (Confluence, GitHub, Notion, etc.)
- Add a memory node to keep conversation history
- Add a UI (Streamlit / React) that sends questions to this agent

You can use this notebook as an **interview demo** or a starting point for a real internal Glean‑like assistant.
